Import libraries

In [ ]:
import numpy as np 
import pandas as pd
import operator
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler
import ppscore as pps
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.svm import SVR
from lightgbm import LGBMClassifier, plot_importance as plot_importance_lgbm
from xgboost import XGBClassifier, XGBRegressor, plot_importance as plot_importance_xgb
from sklearn.model_selection import StratifiedShuffleSplit, RandomizedSearchCV, train_test_split

# 1 - Load data

In [ ]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# 2 - Data exploration

In [ ]:
train_data.head()

In [ ]:
missing_values = (train_data.isnull().sum())
print("Missing values per column:\n", missing_values[missing_values > 0])
print("\n Percentage missing values per column:\n", missing_values[missing_values > 0]/train_data.shape[0]*100)

- 99.5% of PoolQC (pool quality) is missing (aka no pool)
-> there is PoolArea column which will say 0 if no pool
- 93.8% of Alley (type of alley access) is missing (aka no alley access)
- 96.3% of MiscFeature (miscellaneous feature not covered in other categories) is missing
-> there is MiscVal feature that will say 0 if no misc feature
- 80.8% of Fence (fence quality) is missing (aka no fence)
- 47.3% of FireplaceQu (fireplace quality) is missing (aka no fireplace) 

In [ ]:
# Value counts for each column
for column in train_data.columns:
    print("Column: {} \n{} \n".format(column, train_data[column].value_counts()))

In [ ]:
# Number of unique values in each column
unique_cols = {}

for column in train_data.columns:
    unique_cols[column] = len(train_data[column].unique())

for x in sorted(unique_cols.items(), key=operator.itemgetter(1)): 
    print(x)

In [ ]:
train_data.drop(['SalePrice', 'Id'], axis=1).hist(figsize=(18,18))
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
corr = train_data.corr()
sns.heatmap(corr)
plt.show()

In [ ]:
# That was a lot, only top 10:
f, ax = plt.subplots(figsize=(10, 8))
top_corr = train_data[corr.SalePrice.sort_values(ascending=False)[:10].index].corr()
sns.heatmap(top_corr, annot=True)
plt.show()

In [ ]:
# Number of unique values in each top 10 column
top10_cols = ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt']
unique_cols = {}

for column in top10_cols:
    unique_cols[column] = len(train_data[column].unique())

for x in sorted(unique_cols.items(), key=operator.itemgetter(1)): 
    print(x)

In [ ]:
missing_values = (train_data[top10_cols].isnull().sum())
print("Missing values per column:\n", missing_values[missing_values > 0])
print("\n Percentage missing values per column:\n", missing_values[missing_values > 0]/train_data.shape[0]*100)

So no values are missing in the top 10 columns. 

But: (GarageCars, GarageArea) and (TotalBsmtSF, 1stFlrSF) and (TotRmsAbvGrd, 1stFlrSF) are highly correlated. So **GarageCars, TotRmsAbvGrd and TotalBsmtSF will be excluded**. 

In [ ]:
# Remaining columns (features) that are categorical
remaining_df = train_data.drop(["GarageArea", "TotRmsAbvGrd", "TotalBsmtSF", "Id"], axis=1)
remaining_cols_cat = remaining_df.select_dtypes(include='object').columns

# Value counts for each column with categorical value
for column in remaining_cols_cat:
    print("Column: {} \n{} \n".format(column, remaining_df[column].value_counts()))


**Utilities** has only 1 entry where its value is NoSeWa, everything else is AllPub. Kind of the same holds for **Condition2, LandSlope, RoofMatl, GarageCond, GarageQual, Functional, Electrical, Heating, BsmtCond** so they will be exluded.

- 99.5% of PoolQC (pool quality) is missing (aka no pool)
-> there is PoolArea column which will say 0 if no pool
- 93.8% of Alley (type of alley access) is missing (aka no alley access) -> **turn it into yes/no**
- 96.3% of MiscFeature (miscellaneous feature not covered in other categories) is missing
-> there is MiscVal feature that will say 0 if no misc feature
- 80.8% of Fence (fence quality) is missing (aka no fence) -> **turn it into yes/no**
- 47.3% of FireplaceQu (fireplace quality) is missing (aka no fireplace) -> **turn it into yes/no**

# 3 - Feature engineering

In [ ]:
target = train_data.SalePrice
train_data_sub = train_data.drop(["SalePrice", "GarageCars", "TotRmsAbvGrd", "TotalBsmtSF", 
                                "Id", "Utilities", "Condition2", "LandSlope", 
                                "RoofMatl", "GarageCond", "GarageQual", "Functional", 
                                "Electrical", "Heating", "BsmtCond"], 
                               axis=1)

In [ ]:
# # Features that will be one-hot encoded:
# one_hot = [
#             'MSZoning', 'LandContour', 'LotConfig', 'Neighborhood', 
#             'Condition1', 'BldgType',
#             'HouseStyle', 'RoofStyle', 'Exterior1st', 
#             'Exterior2nd', 'MasVnrType', 'Foundation', 
#             'GarageType', 'MiscFeature', 
#             'SaleType', 'SaleCondition'
#             ]

# # Features that will be label encoded:
# label_encode = [
#                 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual',
#                 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual',
#                 'FireplaceQu', 'PoolQC', 'Fence', 'Street', 'Alley',
#                 'GarageFinish', 'MoSold', 'YrSold', 'PavedDrive', 
#                 'CentralAir', 'LotShape', 'MSSubClass', 
#                 ]

# # Features that will stay numerical
# num = [
#         'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
#         '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
#         'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
#         'Fireplaces', 'GarageYrBlt', 'YearBuilt', 'YearRemodAdd',
#         'OverallQual'
#         ]

# Things that will be turned into 1/0
presence = ['MiscVal', 'PoolArea', 'ScreenPorch', '3SsnPorch', 'EnclosedPorch', 'OpenPorchSF', 'WoodDeckSF', 'GarageArea']

# Columns where NaN means the described feature is not present
na = ['Alley', 'BsmtQual', 'BsmtFinType1', 'BsmtFinType2', 'BsmtExposure', 'FireplaceQu', 'GarageType', 'GarageFinish', 'PoolQC', 'Fence', 'MiscFeature']

In [ ]:
# Turn columns into present-or-not columns
for column in presence:
    train_data_sub.loc[train_data_sub[column] > 0, 'Presence'+column] = 1
    train_data_sub.loc[train_data_sub[column] == 0, 'Presence'+column] = 0

# Fill columns where NaN means the described feature is not present
for x in na:
    train_data_sub[na] = train_data_sub[na].fillna("Not present")

    
# Drop columns that are now not needed anymore
train_data_sub = train_data_sub.drop(presence, axis=1)
cat_columns = train_data_sub.select_dtypes(include='object').columns
num_columns = train_data_sub.select_dtypes(exclude='object').columns

Other ideas:
    - Count total number of bathrooms

# 4 - Pipelines and hyper parameter optimization

In [ ]:
# Preprocessing for numerical data
num_transformer_1 = Pipeline(steps=[
                                    ('imputer', SimpleImputer(strategy='mean')),
                                    ('scale', MinMaxScaler())
                                    ])
num_transformer_2 = Pipeline(steps=[
                                    ('imputer', SimpleImputer(strategy='most_frequent')),
                                    ('scale', MinMaxScaler())
                                    ])

# Preprocessing for categorical one_hot data
cat_transformer_1 = Pipeline(steps=[
                                    ('imputer', SimpleImputer(strategy='most_frequent')),
                                    ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                    ])

# Bundle preprocessing for numerical and categorical data
data_transformer_1 = ColumnTransformer(transformers=[
                                                        ('num', num_transformer_1, num_columns),
                                                        ('cat', cat_oh_transformer_1, cat_columns)
                                                    ])
data_transformer_2 = ColumnTransformer(transformers=[
                                                        ('num', num_transformer_2, num_columns),
                                                        ('cat', cat_oh_transformer_1, cat_columns)
                                                    ])

In [ ]:
def find_best_model(data, target):
    X_train, X_valid, y_train, y_valid = train_test_split(data, target, test_size=0.10, random_state=1)

    pipeline = Pipeline(steps=[('preprocess', data_transformer_1), 
                               ('model', SVR())
                              ]) 

    param_grid = [{ 'preprocess': [data_transformer_1, data_transformer_2],
                    'model': [RandomForestRegressor()],
                    #'model__criterion': ["absolute_error"],
                    'model__n_estimators': np.arange(10, 100, 10),
                    'model__max_depth': np.arange(3, 20, 1),
                    'model__max_features': [None, "sqrt", "log2"]
                 },
                 {  'preprocess': [data_transformer_1, data_transformer_2],
                    'model': [Lasso()],
                    'model__alpha': np.arange(0, 200, 5)
                 },
                 {  'preprocess': [data_transformer_1, data_transformer_2],
                    'model': [XGBRegressor()],
                    'model__n_estimators': [int(x) for x in np.linspace(3, 15, num=10)],
                    'model__eta': np.linspace(0.1, 0.9), # learning rate
                    'model__max_depth': [int(x) for x in np.linspace(2, 7, num=5)],
                    'model__gamma': np.linspace(0.1, 1), # min loss reduction required to make further partition on leaf node of tree
                    'model__lambda': np.linspace(0.1, 1) # L2 regularization term on weight
                 }]
                
    best_parameters = RandomizedSearchCV(estimator=pipeline, param_distributions=param_grid, 
                                         cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1,
                                         random_state=1)

    _ = best_parameters.fit(X_train, y_train)

    print("Best data pipeline: {} \n".format(best_parameters.best_estimator_[0]))
    print("Best regressor: {} \n".format(best_parameters.best_estimator_[1]))
    print("Best mean absolute error on training set: {} \n".format(abs(best_parameters.best_score_)))
    
    return X_train, X_valid, y_train, y_valid, best_parameters

In [ ]:
def evaluate_model(best_parameters, X_valid, y_valid):
    predictions = best_parameters.best_estimator_.predict(X_valid)
    result = mean_absolute_error(y_valid, predictions)
    print("Mean absolute error on validation set: {}".format(result))
    return result

In [ ]:
X_train, X_valid, y_train, y_valid, best_parameters = find_best_model(train_data_sub, target)

valid_set_results = evaluate_model(best_parameters, X_valid, y_valid)

# 5 - Predicting target for test set

Prepare test data same way as training data

In [ ]:
# Drop unused features
test_data_sub = test_data.drop(["GarageCars", "TotRmsAbvGrd", "TotalBsmtSF", 
                                "Id", "Utilities", "Condition2", "LandSlope", 
                                "RoofMatl", "GarageCond", "GarageQual", "Functional", 
                                "Electrical", "Heating", "BsmtCond"], 
                               axis=1)

# Turn columns into present-or-not columns
for column in presence:
    test_data_sub.loc[test_data_sub[column] > 0, 'Presence'+column] = 1
    test_data_sub.loc[test_data_sub[column] == 0, 'Presence'+column] = 0

# Fill columns where NaN means the described feature is not present
for x in na:
    test_data_sub[na] = test_data_sub[na].fillna("Not present")

    
# Drop columns that are now not needed anymore
test_data_sub = test_data_sub.drop(presence, axis=1)

Predict target

In [ ]:
# Applying our best model's pipeline and parameters on test set
test_predictions = best_parameters.best_estimator_.predict(test_data_sub)

print(test_predictions)

In [ ]:
# Predictions file that will be submitted
output = pd.DataFrame({'Id': test_data.Id, 'SalePrice': test_predictions})
output.to_csv('housing-prices-submission.csv', index=False)